# Réseau de neurones récurrents

Dans ce cours, nous allons introduire les réseaux de neurones récurrents (RNN) dans le cadre de la prédiction du prochain caractère. Pour ce faire, nous allons nous baser sur l'architecture décrite dans le papier [Recurrent neural network based language model](https://www.fit.vutbr.cz/research/groups/speech/publi/2010/mikolov_interspeech2010_IS100722.pdf) qui présente une version basique de RNN pour la prédiction du prochain caractère.  

La motivation derrière l'utilisation d'un RNN pour cette tâche permet de ne pas avoir à spécifier une taille de contexte pour l'entraînement du modèle contrairement aux deux modèles basées sur des réseaux fully connected que nous avons vu dans les notebooks précédents. 

Les RNN ont pour motivation de garder une information de contexte peu importe la longueur de la séquence. C'est une idée très sensé sur le papier mais il y a de grosses limitations pour l'optimisation.  

<img src="images/rnn.png" alt="rnn" width="250"/>    

Figure extraite de l'article original.


## Fonctionnement de l'architecture RNN

L'architecture de réseau de neurones récurrents se base sur une approche séquentiel. Les caractères vont être passé un par un dans le modèle et la valeur du caractère suivant dépend du "state" gardé en mémoire et de l'élément actuel. Le "state" contient les informations de contexte de tous les caractères précédents.  

Posons le problème mathématiquement : 
Un RNN est constitué de 3 éléments : l'input $x$, le state (hidden layer) $s$ et l'output $y$. On introduit également le temps $t$ qui rajoute la composante temporelle pour le traitement séquentiel.   
$x$ au temps $t$ est alors défini comme :    
$x(t)=w(t) + s(t-1)$ où $w()$ est l'opération de one_hot encoding  
Et ensuite, on estime $s(t)$ et $y(t)$ :    
$s(t)=sigmoid(x(t))$   
$y(t)=softmax(s(t))$    

On peut constater que ce modèle n'a en fait qu'un seul paramètre à ajuster : la dimension de la couche cachée $s$. 

Pour l'initialisation $s(0)$ peut être configuré comme un vecteur de petite valeurs. 

## Implémentation

In [1]:
import torch
import torch.nn as nn

### Dataset

Utiliser un RNN pour générer des prénoms n'est pas très intéressant car la taille de contexte est limitée car les prénoms ne sont jamais très longs. Pour ce type de tâches, il est intéressant d'utiliser un dataset avec un contexte conséquent.   
Pour cela, nous utilons un fichier texte moliere.txt qui regroupe l'intégralité des dialogues des pièces de Molière.   
Ce dataset a été crée à partir des oeuvres complètes de Molière disponibles sur le site [Gutenberg.org](Gutenberg.org). J'ai nettoyé un peu les données pour ne garder que les dialogues.

In [2]:
with open('moliere.txt', 'r', encoding='utf-8') as f:
    text = f.read()
print("Nombre de caractères dans le dataset : ", len(text))

Nombre de caractères dans le dataset :  1687290


C'est un gros dataset, pour avoir un temps de traitement raisonnables, prenons uniquement une partie de ce dataset, par exemple les 100 000 premiers caractères.

In [3]:
text=text[:100000]
print("Nombre de caractères dans le dataset : ", len(text))

Nombre de caractères dans le dataset :  100000


Affichons les 250 premiers caractères.

In [4]:
print(text[:250])

VALÈRE.

Eh bien, Sabine, quel conseil me donnes-tu?

SABINE.

Vraiment, il y a bien des nouvelles. Mon oncle veut résolûment que ma
cousine épouse Villebrequin, et les affaires sont tellement avancées,
que je crois qu'ils eussent été mariés dès aujo


Regardons le nombre de caractères différents : 

In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print("Nombre de caractères différents : ", vocab_size)


 !'(),-.:;?ABCDEFGHIJLMNOPQRSTUVYabcdefghijlmnopqrstuvxyz«»ÇÈÉÊàâæçèéêîïôùû
Nombre de caractères différents :  76


Création d'un mapping de caractère à entiers et inversement

In [6]:
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encore : prend un string et output une liste d'entiers
decode = lambda l: ''.join([itos[i] for i in l]) # decode: prend une liste d'entiers et output un string

Encodons notre dataset en convertissant les string en int puis en le transformant en tenseur pytorch.

In [7]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:250]) # Les 250 premiers caractères encodé

tensor([32, 12, 22, 61, 28, 16,  8,  0,  0, 16, 41,  1, 35, 42, 38, 46,  6,  1,
        29, 34, 35, 42, 46, 38,  6,  1, 49, 53, 38, 44,  1, 36, 47, 46, 51, 38,
        42, 44,  1, 45, 38,  1, 37, 47, 46, 46, 38, 51,  7, 52, 53, 11,  0,  0,
        29, 12, 13, 20, 24, 16,  8,  0,  0, 32, 50, 34, 42, 45, 38, 46, 52,  6,
         1, 42, 44,  1, 56,  1, 34,  1, 35, 42, 38, 46,  1, 37, 38, 51,  1, 46,
        47, 53, 54, 38, 44, 44, 38, 51,  8,  1, 23, 47, 46,  1, 47, 46, 36, 44,
        38,  1, 54, 38, 53, 52,  1, 50, 69, 51, 47, 44, 75, 45, 38, 46, 52,  1,
        49, 53, 38,  1, 45, 34,  0, 36, 47, 53, 51, 42, 46, 38,  1, 69, 48, 47,
        53, 51, 38,  1, 32, 42, 44, 44, 38, 35, 50, 38, 49, 53, 42, 46,  6,  1,
        38, 52,  1, 44, 38, 51,  1, 34, 39, 39, 34, 42, 50, 38, 51,  1, 51, 47,
        46, 52,  1, 52, 38, 44, 44, 38, 45, 38, 46, 52,  1, 34, 54, 34, 46, 36,
        69, 38, 51,  6,  0, 49, 53, 38,  1, 43, 38,  1, 36, 50, 47, 42, 51,  1,
        49, 53,  3, 42, 44, 51,  1, 38, 

On sépare training et validation : 

In [8]:
n = int(0.9*len(data)) # 90% pour le train et 10% pour la validation
train_data = data[:n]
val_data = data[n:]

**Note** : Chaque itération de l'entraînement correspondra à un passage dans l'intégralité du dataset de manière séquentielle.

### Création du modèle 

Il est maintenant temps de créer notre modèle !  

Dans le papier, il est indiqué que l'entrée du modèle (le caractère) est encodé en one hot et qu'il est ensuite sommé avec le state à $t-1$. On va donc avoir besoin de deux couches fully connected, la première pour transformer l'entrée $x(t)$ en  state au temps t$s(t)$ et la seconde pour transformer $s(t)$ en $y(t)$, notre prédiction. 

<img src="images/rnn_math.png" alt="rnn_math" width="250"/>    

Equation extraite de l'article original. $f$ est la fonction sigmoid et $g$ la softmax.

**Note** : L'[article](https://www.fit.vutbr.cz/research/groups/speech/publi/2010/mikolov_interspeech2010_IS100722.pdf) est très accessible et concis, je vous invite à le lire. 


In [9]:
class rnn(nn.Module): 
  def __init__(self,hidden_dim,vocab_size) -> None:
    super(rnn, self).__init__()
    self.hidden_to_hidden=nn.Linear(hidden_dim+vocab_size, hidden_dim)
    self.hidden_to_output=nn.Linear(hidden_dim, vocab_size)
    self.vocab_size=vocab_size
    self.hidden_dim=hidden_dim
    self.sigmoid=nn.Sigmoid() 
    
  # Le réseau prend en entrée le caractère actuel et le state précédent
  def forward(self, x,state):
    # On one-hot encode le caractère
    x = torch.nn.functional.one_hot(x, self.vocab_size).float()
    if state is None:
      # Si on a pas de state (début de la séquence), on initialise le state avec des petites valeurs aléatoires
      state = torch.randn(self.hidden_dim) * 0.1
    x = torch.cat((x, state), dim=-1)  # Concaténation de x et du state
    state = self.sigmoid(self.hidden_to_hidden(x)) # Calcul du nouveau state
    output = self.hidden_to_output(state) # Calcul de l'output
    # On renvoie l'output et le state pour le prochain pas de temps
    return output, state.detach() # detach() pour éviter de propager le gradient dans le state

Définissons nos paramètres d'entraînement : 

In [10]:
epochs = 10
lr=0.1
hidden_dim=128
model=rnn(hidden_dim,vocab_size)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

/home/aquilae/anaconda3/envs/dev/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Il est temps d'entraîner le modèle !!

In [11]:
for epoch in range(epochs):
    state=None
    running_loss = 0
    n=0
    for i in range(len(train_data)-1):
        x = train_data[i]
        y = train_data[i+1]
        optimizer.zero_grad()
        y_pred,state = model.forward(x,state)
        loss = criterion(y_pred, y)
        running_loss += loss.item()
        n+=1
        loss.backward()
        optimizer.step()

    print("Epoch: {0} \t Loss: {1:.8f}".format(epoch, running_loss/n))

Epoch: 0 	 Loss: 2.42064296
Epoch: 1 	 Loss: 2.02932206
Epoch: 2 	 Loss: 1.91027851
Epoch: 3 	 Loss: 1.84189548
Epoch: 4 	 Loss: 1.79332830
Epoch: 5 	 Loss: 1.75458955
Epoch: 6 	 Loss: 1.72111627
Epoch: 7 	 Loss: 1.69267295
Epoch: 8 	 Loss: 1.66852771
Epoch: 9 	 Loss: 1.64716606


Maintenant que le modèle est entraîné, on va pouvoir générer du Molière !!!

In [23]:
import torch.nn.functional as F 
moliere='.'
sequence_length=250
state=None
for i in range(sequence_length):
    x = torch.tensor(encode(moliere[-1]), dtype=torch.long).squeeze()
    y_pred,state = model.forward(x,state)
    probs=F.softmax(torch.squeeze(y_pred), dim=0)
    sample=torch.multinomial(probs, 1)
    #print(sample)
    moliere+=itos[sample.item()]
print(moliere)

.

LÉLIE.

Vous dout asspagé bodrt, ouis horrite.

LÉLIE.

Oui mageît;
Cons tourre un vous sous l'avousont avic'obal nant fivectie,
Mans cet avois le soup jommoye,
D'achiviller cet il vivons ce vire.

MASCARILLE.

Adieur tagours.

LÉLIE.

Le forle;
A 


Ce n'est pas très convaincant ... Mais on reconnaît quand même quelques mots et un agencement des phrases similaire au fichier "moliere.txt". Ce n'est finalement pas si mal pour un réseau récurrent d'une seule couche. 

**Comment améliorer nos résultats ?** : Pour améliorer les résultats, il y a plusieurs options possibles :
- On peut augmenter le nombre de couche récurrente ou augmenter la dimension de la couche cachée.
- On peut utiliser un embedding plutôt qu'un one hot encoding
- On peut utiliser d'autres variantes de RNN comme [LSTM](https://arxiv.org/pdf/1308.0850) ou [GRU](https://arxiv.org/abs/1409.1259).